In [ ]:
####Duration time separation model####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import math
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [ ]:
####指数-正規混合モデルの推定####
##データの設定
s = 2
hh = 10000

In [ ]:
##パラメータの生成
#セグメント割当の生成
pi = np.array([0.6])
z = np.random.binomial(1, pi, hh)
index_z1 = np.array(np.where(z==1)[0], dtype="int")
index_z2 = np.array(np.where(z==0)[0], dtype="int")
hh1 = index_z1.shape[0]
hh2 = index_z2.shape[0]

#モデルパラメータの生成
Sigma = np.array([1.25])
mu = np.array([3.0])
theta = np.array([0.5])

In [ ]:
#応答変数の生成
y = np.repeat(0.0, hh)
y[index_z1] = np.random.normal(mu, Sigma, hh1)
y[index_z2] = np.random.exponential(theta, hh2)
y[y < 0] = 0.0

In [ ]:
#応答変数の可視化
fig = plt.figure(figsize=(13.5, 4.0))
ax1 = fig.add_subplot(1, 2, 1); ax2 = fig.add_subplot(1, 2, 2)
ax1.hist(y[index_z1], bins=25, range=[0.0, np.max(y)])
ax1.hist(y[index_z2], bins=25, range=[0.0, np.max(y)])
ax2.hist(y, bins=25, range=[0.0, np.max(y)])

In [ ]:
####EMアルゴリズムでパラメータを推定####
##観測データの尤度と潜在変数zを計算する関数
def ollz(mu, Sigma, theta, pi, y):
    #モデルごとの尤度を定義
    Lho1 = scipy.stats.norm.pdf(y, mu, Sigma)
    Lho2 = scipy.stats.expon.pdf(y, scale=1/theta)

    #観測データの対数尤度
    Lho = np.hstack((pi*Lho1[:, np.newaxis], (1-pi)*Lho2[:, np.newaxis]))
    LLobz = np.sum(np.log(np.sum(Lho, axis=1)))

    #潜在変数zの定義
    z = Lho / np.sum(Lho, axis=1)[:, np.newaxis]
    return LLobz, z

In [ ]:
##EMアルゴリズムの設定
iters = 0
rp = 200   #最大繰り返し数
LL = -1000000000   #対数尤度の初期値
dl = 100   #EMアルゴリズムの対数尤度の差の初期値
tol = 0.1
maxit = 20   #準ニュートン法のステップ数

In [ ]:
##EMアルゴリズムの初期値
#混合率と潜在変数の初期値
pi = np.array([1/s])
zpt = np.repeat(pi, hh)

#パラメータの初期値
Sigma = np.random.uniform(1.0, 2.0, 1)
mu = np.random.uniform(1.5, 3.0, 1)
theta = np.random.uniform(0.25, 0.5, 1)

In [ ]:
#観測データの対数尤度と潜在変数
oll = ollz(mu, Sigma, theta, pi, y)
zpt = oll[1]
LL1 = oll[0]

In [ ]:
##EMアルゴリズムでパラメータを推定
while abs(dl) >= tol:
    
    #Mステップで完全データモデルの最尤推定
    m = np.sum(zpt, axis=0)
    u = zpt * y[:, np.newaxis]
    mu = np.array([np.sum(u[:, 0]) / np.sum(m[0])])
    Sigma = np.sqrt(np.sum(np.power(zpt[:, 0]*(y - mu), 2)) / m[0])
    theta = np.mean(u[:, 1])

    #混合率を更新
    pi = m[0] / hh

    #Eステップで観測データの対数尤度と潜在変数zを更新
    oll = ollz(mu, Sigma, theta, pi, y)
    zpt = oll[1]
    LL = oll[0]

    #EMアルゴリズムのパラメータを更新
    iters = iters + 1
    dl = LL - LL1
    LL1 = LL
    print(LL)

In [ ]:
####ガンマ-ガンマ混合モデルの推定####
##データの設定
s = 2
hh = 100000

##パラメータの生成
#セグメント割当の生成
pi = np.array([0.6])
z = np.random.binomial(1, pi, hh)
index_z1 = np.array(np.where(z==1)[0], dtype="int")
index_z2 = np.array(np.where(z==0)[0], dtype="int")
hh1 = index_z1.shape[0]
hh2 = index_z2.shape[0]

#モデルパラメータの生成
kappa = np.array([4.0, 1.0])
theta = np.array([1.0, 2.0])

In [ ]:
#応答変数の生成
y = np.repeat(0.0, hh)
y[index_z1] = np.random.gamma(kappa[0], 1/theta[0], hh1)
y[index_z2] = np.random.gamma(kappa[1], 1/theta[1], hh2)

In [ ]:
#応答変数の可視化
fig = plt.figure(figsize=(13.5, 4.0))
ax1 = fig.add_subplot(1, 2, 1); ax2 = fig.add_subplot(1, 2, 2)
ax1.hist(y[index_z1], bins=25, range=[0.0, np.max(y)])
ax1.hist(y[index_z2], bins=25, range=[0.0, np.max(y)])
ax2.hist(y, bins=25, range=[0.0, np.max(y)])

In [ ]:
####EMアルゴリズムでパラメータを推定####
##パラメータを推定するための関数
#観測データの尤度と潜在変数zを計算する関数
def ollz(kappa, theta, pi, y):
    #モデルごとの尤度を定義
    Lho1 = scipy.stats.gamma.pdf(y, kappa[0], scale=1/theta[0])
    Lho2 = scipy.stats.gamma.pdf(y, kappa[1], scale=1/theta[1])
    
    #観測データの対数尤度
    Lho = np.hstack((pi*Lho1[:, np.newaxis], (1-pi)*Lho2[:, np.newaxis]))
    LLobz = np.sum(np.log(np.sum(Lho, axis=1)))

    #潜在変数zの定義
    z = Lho / np.sum(Lho, axis=1)[:, np.newaxis]
    return LLobz, z

#完全データの対数尤度関数
def cll(theta, z, y):
    #パラメータの定義
    alpha = np.exp(theta[:s])
    beta = np.exp(theta[s:])

    #重み付き対数尤度の和
    y_vec = y[:, np.newaxis]
    LL = np.sum(z * (alpha*np.log(beta) - scipy.special.loggamma(alpha) + (alpha-1)*np.log(y_vec) - beta*y_vec))
    return -LL 

#完全データの対数微分関数
def dll(theta, z, y):
    #パラメータの定義
    alpha = np.exp(theta[:s])
    beta = np.exp(theta[s:])

    #勾配ベクトル
    y_vec = y[:, np.newaxis]
    sc1 = np.sum(z * (np.log(beta) - scipy.special.digamma(alpha) + np.log(y_vec)), axis=0)
    sc2 = np.sum(z * (alpha*(1/beta) - y_vec), axis=0)
    return -np.append(sc1, sc2)

In [ ]:
##EMアルゴリズムの設定
iters = 0
rp = 200   #最大繰り返し数
LL = -1000000000   #対数尤度の初期値
dl = 100   #EMアルゴリズムの対数尤度の差の初期値
tol = 0.1
maxit = 20   #準ニュートン法のステップ数

In [ ]:
##EMアルゴリズムの初期値
#混合率と潜在変数の初期値
pi = np.array([1/s])
zpt = np.repeat(pi, hh)

#パラメータの初期値
kappa = np.random.uniform(0.5, 3.0, s)
theta = np.random.uniform(0.5, 2.0, s)

In [ ]:
#観測データの対数尤度と潜在変数
oll = ollz(kappa, theta, pi, y)
zpt = oll[1]
LL1 = oll[0]

In [ ]:
##EMアルゴリズムでパラメータを推定
while abs(dl) >= tol:
    
    #Mステップで完全データ対数尤度を最大化
    res = optimize.minimize(cll, np.log(np.append(kappa, theta)), jac=dll, method="BFGS", args=(zpt, y), options={"disp": False})
    kappa = np.exp(res.x[:s])
    theta = np.exp(res.x[s:])

    #混合率を更新
    pi = m[0] / hh

    #Eステップで観測データの対数尤度と潜在変数zを更新
    oll = ollz(kappa, theta, pi, y)
    zpt = oll[1]
    LL = oll[0]

    #EMアルゴリズムのパラメータを更新
    iters = iters + 1
    dl = LL - LL1
    LL1 = LL
    print(LL)